In [5]:
import dolfin as d
from smart import mesh_tools
import pathlib
import numpy as np
import sys
sys.path.append("/root/shared/gitrepos/smart-nanopillars/utils")
import spread_cell_mesh_generation as mesh_gen
indentation = 2.8
nucScaleFactor = 0.8
nanopillars = [0.5, 3.0, 3.5]

In [ ]:
loaded = mesh_tools.load_mesh(pathlib.Path("/root/shared/gitrepos/smart-comp-sci-data/meshes/nanopillars_new_moretheta/nanopillars_indent2.8") / "spreadCell_mesh.h5")
zMax = max(loaded.mesh.coordinates()[:,2])
nanopillar_rad, nanopillar_height, nanopillar_spacing = nanopillars[:]
u_nuc, a, b = mesh_gen.get_u_nuc(nanopillar_height, zMax, 
                                           nucScaleFactor, indentation, nanopillars)
xSteric = 0.05
xCurv = 0.2
nanopillar_rad += 2*xSteric # to avoid collision with PM
aInner0, bInner0, r0Inner, z0Inner = mesh_gen.get_inner_param(nanopillar_height, zMax, nucScaleFactor, indentation)
a_nuc = mesh_gen.compute_stretch()
d.File("u_nuc.pvd") << u_nuc

In [ ]:
hEdge = 0.5
hInnerEdge = 0.5
hNP = hEdge * 0.01
cell_mesh, facet_markers, cell_markers, substrate_markers, curv_markers, u_nuc, a_nuc = mesh_gen.create_3dcell(
                                                                    contactRad=15.5,
                                                                    hEdge=hEdge, hInnerEdge=hInnerEdge,
                                                                    hNP=hNP,
                                                                    thetaExpr="1",
                                                                    nanopillars=nanopillars,
                                                                    return_curvature=True,
                                                                    sym_fraction=1/8,
                                                                    nuc_compression=indentation)
d.File("a_nuc.pvd") << a_nuc
d.File("nuc_mesh.pvd") << d.create_meshview(facet_markers, 12)

In [9]:
loaded = mesh_tools.load_mesh(pathlib.Path("/root/shared/gitrepos/smart-comp-sci-data/meshes/nanopillars_new_moretheta/nanopillars_indent2.8") / "spreadCell_mesh.h5")
nanopillar_rad, nanopillar_height, nanopillar_spacing = nanopillars[:]
facet_markers = loaded.mf_facet
zMax = max(loaded.mesh.coordinates()[:,2])
nanopillar_rad, nanopillar_height, nanopillar_spacing = nanopillars[:]
u_nuc, a, b = mesh_gen.get_u_nuc(nanopillar_height, zMax, 
                                           nucScaleFactor, indentation, nanopillars)
xSteric = 0.05
xCurv = 0.2
nanopillar_rad += 2*xSteric # to avoid collision with PM
aInner0, bInner0, r0Inner, z0Inner = mesh_gen.get_inner_param(nanopillar_height, zMax, nucScaleFactor, indentation)
# define nanopillar locations
xMax = np.ceil(a / nanopillar_spacing) * nanopillar_spacing
xNP = np.arange(-xMax, xMax+1e-12, nanopillar_spacing)
yNP = np.arange(-xMax, xMax+1e-12, nanopillar_spacing)
xNP, yNP = np.meshgrid(xNP, yNP)
xNP = xNP.flatten()
yNP = yNP.flatten()

mesh_cur = d.create_meshview(facet_markers, 12)
Vcur = d.FunctionSpace(mesh_cur, "P", 1)
curv_fcn = d.Function(Vcur)

# Vcur = u_nuc.function_space()
# curv_fcn = d.Function(Vcur)

def calc_curv_NP(x,y,xNP,yNP,zNP,radNP,zTest):
    if len(zNP) == 1:
        zNP = zNP * np.ones_like(xNP)
    else:
        assert len(zNP) == len(xNP)
    dist_vals = np.sqrt(np.power(xNP-x, 2) + np.power(yNP-y, 2)) 
    uVal = 0
    alpha = np.sqrt(1 - (x**2+y**2)/a**2)
    if alpha == 0:
        curv_val = b*(a**2 + b**2) / (2*a*b**3)
        return curv_val
    zCur = z0Inner - b*alpha
    hex = b*x/(a**2 * alpha)
    hey = b*y/(a**2 * alpha)
    hexy = b*x*y / (a**4 * alpha**3)
    hexx = b*x**2 / (a**4 * alpha**3) + b / (a**2 * alpha)
    heyy = b*y**2 / (a**4 * alpha**3) + b / (a**2 * alpha)
    hx = hex
    hy = hey
    hxy = hexy
    hxx = hexx
    hyy = heyy
    for i in range(len(xNP)):
        uRef = zNP[i] - zCur
        if zTest < zNP[i]+1e-6:
            dxCur = x - xNP[i]
            dyCur = y - yNP[i]
            rlocal = np.sqrt(dxCur**2 + dyCur**2)
            drCur = rlocal - radNP
            sigma1 = np.exp(-(drCur/0.2)**2)
            if dist_vals[i] < radNP:
                curv_val = 0
                return curv_val
            else:
                hx += -hex*sigma1 - 2*uRef*drCur*dxCur*sigma1/(0.2**2 * rlocal)
                hy += -hey*sigma1 - 2*uRef*drCur*dyCur*sigma1/(0.2**2 * rlocal)
                xyCur = (-hexy*sigma1 + 2*hex*drCur*dyCur*sigma1/(0.2**2 * rlocal) + 2*hey*drCur*dxCur*sigma1/(0.2**2 * rlocal) +
                        (2*sigma1*uRef*drCur*dxCur*dyCur/(0.2**2 * rlocal**2))*(2*drCur/0.2**2 + 1/rlocal) - 
                        2*uRef*dxCur*dyCur*sigma1/(0.2**2 * rlocal**2))
                xyCurAlt = (-hexy*sigma1 + 2*hey*drCur*dxCur*sigma1/(0.2**2 * rlocal) + 2*hex*drCur*dyCur*sigma1/(0.2**2 * rlocal) +
                        (2*sigma1*uRef*drCur*dxCur*dyCur/(0.2**2 * rlocal**2))*(2*drCur/0.2**2 + 1/rlocal) - 
                        2*uRef*dxCur*dyCur*sigma1/(0.2**2 * rlocal**2))
                assert np.isclose(xyCur, xyCurAlt), f"{xyCur} vs {xyCurAlt}"
                hxy += xyCurAlt
                hxx += (-hexx*sigma1 + 4*hex*drCur*dxCur*sigma1/(0.2**2 * rlocal) + 
                        (2*sigma1*uRef*drCur*dxCur**2/(0.2**2 * rlocal**2))*(2*drCur/0.2**2 + 1/rlocal) - 
                        2*uRef*dxCur**2*sigma1/(0.2**2 * rlocal**2) - 2*uRef*drCur*sigma1/(0.2**2 * rlocal))
                hyy += (-heyy*sigma1 + 4*hey*drCur*dyCur*sigma1/(0.2**2 * rlocal) + 
                        (2*sigma1*uRef*drCur*dyCur**2/(0.2**2 * rlocal**2))*(2*drCur/0.2**2 + 1/rlocal) - 
                        2*uRef*dyCur**2*sigma1/(0.2**2 * rlocal**2) - 2*uRef*drCur*sigma1/(0.2**2 * rlocal))
    num = (1+hy**2)*hxx - 2*hx*hy*hxy + (1+hx**2)*hyy
    den = 2*(1 + hx**2 + hy**2)**(3/2)
    curv_val = num/den   
    return curv_val

uvec = curv_fcn.vector()[:]
coords = Vcur.tabulate_dof_coordinates()
for i in range(len(coords)):
    uvec[i] = calc_curv_NP(coords[i,0],coords[i,1],xNP,yNP,
                           [nanopillar_height+0.2],nanopillar_rad, coords[i,2])
curv_fcn.vector().set_local(uvec)
curv_fcn.vector().apply("insert")
d.File("curv_2.8.pvd") << curv_fcn

Calculate average curvature near nanopillar.

In [13]:
subdomain_mf = d.MeshFunction("size_t", mesh_cur, 2, 0)
zMaxNuc = max(mesh_cur.coordinates()[:,2])
class AnalyzeRegion(d.SubDomain):
    def inside(self, x, on_boundary):
        r_cur = np.sqrt(x[0]**2 + x[1]**2)
        return (r_cur < nanopillar_rad+0.2
                and x[2] < zMaxNuc-b/2)
region = AnalyzeRegion()
region.mark(subdomain_mf, 1)
dx_cur = d.Measure("dx", domain=mesh_cur, subdomain_data=subdomain_mf)
d.assemble(curv_fcn*dx_cur(1))

-3.405509574643292

Check curvature expressions in sympy

In [ ]:
import sympy as sym
aInner0, bInner0, r0Inner, z0Inner = mesh_gen.get_inner_param(nanopillar_height, zMax, nucScaleFactor, indentation)
a = aInner0
b = bInner0

def calc_curv_NP_sym(x,y,xNP,yNP,zNP,radNP,zTest):
    if len(zNP) == 1:
        zNP = zNP * np.ones_like(xNP)
    else:
        assert len(zNP) == len(xNP)
    dist_vals = np.sqrt(np.power(xNP-x, 2) + np.power(yNP-y, 2)) 
    uVal = 0

    xSym, ySym = sym.symbols('x,y')
    he = z0Inner - b*sym.sqrt(1 - (xSym**2 + ySym**2)/a**2)
    hexSym = sym.diff(he, xSym)
    heySym = sym.diff(he, ySym)
    hexySym = sym.diff(hexSym, ySym)
    hexxSym = sym.diff(hexSym, xSym)
    heyySym = sym.diff(heySym, ySym)


    alpha = np.sqrt(1 - (x**2+y**2)/a**2)
    zCur = z0Inner - b*alpha
    subsDict = {xSym:x, ySym:y}
    hex = float(hexSym.subs(subsDict))
    hey = float(heySym.subs(subsDict))
    hexy = float(hexySym.subs(subsDict))
    hexx = float(hexxSym.subs(subsDict))
    heyy = float(heyySym.subs(subsDict))
    hx = hex
    hy = hey
    hxy = hexy
    hxx = hexx
    hyy = heyy
    for i in range(len(xNP)):
        uRef = zNP[i] - zCur
        if zTest < zNP[i]+1e-6:
            dxCur = x - xNP[i]
            dyCur = y - yNP[i]
            rlocal = np.sqrt(dxCur**2 + dyCur**2)
            drCur = rlocal - radNP
            sigma1 = np.exp(-(drCur/0.2)**2)
            if dist_vals[i] < radNP:
                curv_val = 0
                return curv_val
            else:
                rlocalSym = sym.sqrt((xSym-xNP[i])**2 + (ySym-yNP[i])**2)
                hSym = (zNP[i] - he) * sym.exp(-((rlocalSym - radNP)/0.2)**2)
                hxSym = sym.diff(hSym, xSym)
                hySym = sym.diff(hSym, ySym)
                hxySym = sym.diff(hxSym, ySym)
                hxxSym = sym.diff(hxSym, xSym)
                hyySym = sym.diff(hySym, ySym)
                hx += float(hxSym.subs(subsDict))
                hy += float(hySym.subs(subsDict))
                hxy += float(hxySym.subs(subsDict))
                hxx += float(hxxSym.subs(subsDict))
                hyy += float(hyySym.subs(subsDict))
                if np.abs(float(hxSym.subs(subsDict))) > 0.001:
                    print("pause") 
    num = (1+hy**2)*hxx - 2*hx*hy*hxy + (1+hx**2)*hyy
    den = 2*(1 + hx**2 + hy**2)**(3/2)
    curv_val = num/den   
    return curv_val

xVals = np.linspace(-a/5, a/5, 21)
yVals = np.linspace(-a/5, a/5, 21)
xGrid, yGrid = np.meshgrid(xVals, yVals)
curv = np.zeros_like(xGrid)
for i in range(len(xVals)):
    for j in range(len(yVals)):
        curv1 = calc_curv_NP_sym(xGrid[i][j],yGrid[i][j],xNP,yNP,
                           [nanopillar_height + 0.2],nanopillar_rad, 0)
        curv2 = calc_curv_NP(xGrid[i][j],yGrid[i][j],xNP,yNP,
                           [nanopillar_height + 0.2],nanopillar_rad, 0)
        assert np.isclose(curv1, curv2)
        curv[i][j] = curv1
        print(f"{i}, {j}")
from matplotlib import pyplot as plt
ax = plt.axes(projection ='3d')
surf = ax.plot_surface(xGrid, yGrid, curv,
                cmap = plt.get_cmap('viridis'),
                edgecolor = 'none')
ax.view_init(90, 0)
plt.colorbar(surf)